In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00 debug #1 run_with_timeout_async / timeout: 60
00:00:00 debug #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "pwsh -c "../../../../scripts/invoke-dib.ps1 test.dib"", None, None)
00:00:01 verbose #2 > core.Invoke-Dib / Get-Location: C:\home\git\polyglot\apps\spiral\temp\test / path: test.dib / _args: null
00:00:03 verbose #3 >
00:00:03 verbose #4 > ── markdown ────────────────────────────────────────────────────────────────────
00:00:03 verbose #5 > ╭──────────────────────────────────────────────────────────────────────────────╮
00:00:03 verbose #6 > │ # test (Polyglot)                                                            │
00:00:03 verbose #7 > ╰──────────────────────────────────────────────────────────────────────────────╯
00:00:03 verbose #8 >
00:00:03 verbose #9 > ── spiral ──────────────────────────────────────────────────────────────────────
00:00:03 verbose #10 > // // test
00:00:03 verbose #11 >
00:00:03 verbose #1

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(GetExecutableSuffix) test.dib spi } | Invoke-Block

00:00:00 debug #1 writeDibCode / output: Spi / path: test.dib
00:00:00 debug #2 parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 debug #1 run_with_timeout_async / timeout: 60
00:00:00 debug #2 run_with_timeout_async / timeout: 60
00:00:00 debug #1 buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:00 debug #2 buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:01 debug #3 sendJson / port: 13805 / json: {"FileOpen":{"spiText":"// // # test (Polyglot)\n\nnominal i = ()\nnominal e = ()\nnominal s = ()\nnominal n = ()\nnominal t = ()\nnominal f = ()\nnominal j = ()\nnominal p = ()\n\nunion sensing =\n    | Si : s * i\n    | Se : s * e\n\nunion intuition =\n    | Ni : n * i\n    | Ne : n * e\n\nunion thinking =\n    | Ti : t * i\n    | Te : t * e\n\nunion feeling =\n    | Fi : f * i\n    | Fe : f * e\n\nunion function_stack =\n    | FS : sensing * intuition * thinking * feeling\n\nunion personality_type =\n    | ISTJ : i * s * t * j * function_stack\n    | ISFJ : i * s * f * j * function_stack\n    | INFJ : i * n * f * j * function_stack\n   

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @simra
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 179ms

Started Fable compilation...

Fable compilation finished in 1470ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling syn v2.0.59
   Compiling thiserror-impl v1.0.58
   Compiling tracing-attributes v0.1.27
   Compiling thiserror v1.0.58
   Compiling tracing v0.1.40
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
   --> apps\spiral\temp\test\./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^
    |
    = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:41:8
   |
41 | struct Cart {
   |        ^^^^
   |
   = note: `#[warn(dead_code)]` on by default

  --> apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&mut self, item: &Item) {
   |        ^^^^^^^^^^^

   --> apps\spiral\temp\test\./main.rs:124:4
    |
124 | fn parse_comment(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling thiserror v1.0.58
   Compiling tracing v0.1.40
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished `release` profile [optimized] target(s) in 27.36s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\spiral_temp_test-d5c9130a5cb53f2c.exe)

running 3 tests
2024-04-15T13:04:35.558939Z  INFO spiral_temp_test: input=StringLiteral("/TNp^#n")
2024-04-15T13:04:35.560589Z  INFO spiral_temp_test: input=Identifier("DdkEew169Jr5c")
2024-04-15T13:04:35.560720Z  INFO spiral_temp_test: input=Comment("*<'wt\"&QE M</'ah?M'\\`W'\\/2<")
2024-04-15T13:04:35.560810Z  INFO spiral_temp_test: input=Identifier("qOlnIMvo576pPD02")
2024-04-15T13:04:35.560889Z  INFO spiral_temp_test: input=StringLiteral("z4~%9=%'.C'")
2024-04-15T13:04:35.560962Z  INFO spiral_temp_test: input=StringLiteral("D',T;{`")
2024-04-15T13:04:35.561054Z  INFO spiral_temp_test: input=StringLiteral(".S*/=~[%V`T1.xRZmE")
2024-04-15T13:04:35.561161Z  INFO spiral_tem

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
